In [1]:
] add CSV

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
] add DataFrames

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
using CSV # CSV is a common data file format. For high performance computing in physics htf5 format is also used.

In [4]:
using DataFrames

In [5]:
alldata = CSV.read("data.csv", DataFrame)

Row,row,x,y,sigma_y,sigma_x,rho_xy
,Int64,Int64,Int64,Int64,Int64,Float64
1,1,201,592,61,9,-0.84
2,2,244,401,25,4,0.31
3,3,47,583,38,11,0.64
4,4,287,402,15,7,-0.27
5,5,203,495,21,5,-0.33
6,6,58,173,15,9,0.67
7,7,210,479,27,4,-0.02
8,8,202,504,14,4,-0.05
9,9,198,510,30,11,-0.84


In [6]:
alldata.x

20-element Vector{Int64}:
 201
 244
  47
 287
 203
  58
 210
 202
 198
 158
 165
 201
 157
 131
 166
 160
 186
 125
 218
 146

In [7]:
size(alldata)

(20, 6)

In [8]:
size(alldata,1)

20

In [9]:
data = alldata[5:size(alldata,1), :] # alldata[5:end, :] 

Row,row,x,y,sigma_y,sigma_x,rho_xy
,Int64,Int64,Int64,Int64,Int64,Float64
1,5,203,495,21,5,-0.33
2,6,58,173,15,9,0.67
3,7,210,479,27,4,-0.02
4,8,202,504,14,4,-0.05
5,9,198,510,30,11,-0.84
6,10,158,416,16,7,-0.69
7,11,165,393,14,5,0.3
8,12,201,442,25,5,-0.46
9,13,157,317,52,5,-0.03


In [10]:
using WGLMakie

In [11]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
f

In [12]:
m = 2
b = 50

50

In [13]:
xx = LinRange(50, 250, 25)

25-element LinRange{Float64, Int64}:
 50.0,58.3333,66.6667,75.0,83.3333,…,216.667,225.0,233.333,241.667,250.0

In [14]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)
yy = m .*xx .+ b
lines!(xx, yy)
f

In [15]:
] add Optim

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [16]:
using Optim

In [17]:
function abs_diff(x, mean)
    return sum(abs.(x .- mean))
end

abs_diff (generic function with 1 method)

In [18]:
function line_abs_diff(x, b, m, y)
    y_pred = b .+ x .* m
    return abs_diff(y_pred, y)
end

line_abs_diff (generic function with 1 method)

In [19]:
function objective_abs(params)
    b = params[1]
    m = params[2]
    return line_abs_diff(data.x, b, m, data.y)
end

objective_abs (generic function with 1 method)

In [20]:
function objective(params)
    b = params[1]
    m = params[2]
    @show b, m # debugging command, prints (b,m) values as a tuple
    return abs(b - 3.4) + abs(m - 17) 
end

objective (generic function with 1 method)

In [21]:
function objective_2(params)
    b = params[1]
    m = params[2]
    
    y_pred = b .+ m .* data.x
    return sum(abs.(data.y - y_pred)) 
end

objective_2 (generic function with 1 method)

In [22]:
result = optimize(objective_2, [b,m] .+ 0.) #

 * Status: success

 * Candidate solution
    Final objective value:     3.765701e+02

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    97
    f(x) calls:    190


In [23]:
result = optimize(objective, [b,m] .+ 0.) # adding 0. turns the integer to floats

(b, m) = (50.0, 2.0)
(b, m) = (75.025, 2.0)
(b, m) = (50.0, 3.025)
(b, m) = (24.974999999999994, 3.0250000000000004)
(b, m) = (-0.05000000000001137, 3.5375000000000005)
(b, m) = (-0.05000000000001137, 4.5625)
(b, m) = (-25.075000000000017, 5.84375)
(b, m) = (-50.10000000000002, 5.075000000000001)
(b, m) = (24.974999999999994, 3.5375000000000005)
(b, m) = (-25.075000000000017, 4.562500000000001)
(b, m) = (12.462499999999991, 3.7937500000000006)
(b, m) = (-12.562500000000014, 4.30625)
(b, m) = (6.20624999999999, 3.921875000000001)
(b, m) = (6.20624999999999, 4.9468749999999995)
(b, m) = (9.33437499999999, 5.651562499999999)
(b, m) = (12.462499999999991, 4.30625)
(b, m) = (3.0781249999999893, 4.4984375)
(b, m) = (3.0781249999999893, 5.523437499999999)
(b, m) = (1.514062499999989, 6.324218749999998)
(b, m) = (-0.05000000000001137, 5.074999999999998)
(b, m) = (4.64218749999999, 4.97890625)
(b, m) = (1.514062499999989, 5.042968749999998)
(b, m) = (3.8601562499999895, 4.994921874999999)
(b, m

 * Status: success

 * Candidate solution
    Final objective value:     2.553867e-08

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    77
    f(x) calls:    150


In [24]:
b_abs, m_abs = Optim.minimizer(result)

2-element Vector{Float64}:
  3.3999999842335686
 16.999999990227757

In [25]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)
yy_abs = m_abs.*xx .+ b_abs
lines!(xx, yy_abs, color=:green)
yy = m.*xx .+ b
lines!(xx, yy, color=:orange)
f

In [26]:
result = optimize(objective_abs, [b,m] .+ 0.)

 * Status: success

 * Candidate solution
    Final objective value:     3.765701e+02

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    97
    f(x) calls:    190


In [27]:
b_abs,m_abs = Optim.minimizer(result)

2-element Vector{Float64}:
 53.74766339105346
  2.056074767629348

In [28]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
lines!(xx, b_abs .+ m_abs .* xx)
f

In [29]:
function gaussian_log_likelihood(x, mean, sigma)
    return sum(-log.(sigma * sqrt(2*π)) - 0.5 * (x - mean).^2 ./ sigma.^2)
end

gaussian_log_likelihood (generic function with 1 method)

In [30]:
gaussian_log_likelihood(data.y, m .* data.x .+ b, data.sigma_y)

-87.19281204285342

In [31]:
function objective(params)
    b = params[1]
    m = params[2]
    y_pred = b .+ m .* data.x
    return -gaussian_log_likelihood(data.y .+ 0., y_pred, data.sigma_y .+ 0.)
end

objective (generic function with 1 method)

In [32]:
[b,m].+0.

2-element Vector{Float64}:
 50.0
  2.0

In [33]:
result = optimize(objective, [b,m] .+ 0.)

 * Status: success

 * Candidate solution
    Final objective value:     7.430617e+01

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    33
    f(x) calls:    69


In [34]:
b_opt,m_opt = Optim.minimizer(result)

2-element Vector{Float64}:
 34.0476433503552
  2.239922318298898

In [35]:
function objective_2(x, b, m, y, sigma_y)
    y_pred = b .+ m .* x
    return -gaussian_log_likelihood(y, y_pred, sigma_y)
end

objective_2 (generic function with 2 methods)

In [36]:
result = optimize(p -> objective_2(data.x, p[1], p[2], data.y, data.sigma_y),
                  [b,m] .+ 0.);
Optim.converged(result)

true

In [37]:
b_opt,m_opt = Optim.minimizer(result)

2-element Vector{Float64}:
 34.0476433503552
  2.239922318298898

In [38]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
lines!(xx, b_opt .+ m_opt .* xx, )
lines!(xx, b_abs .+ m_abs .* xx, color=:red)
f

In [39]:
append!([1,2],[3,4])

4-element Vector{Int64}:
 1
 2
 3
 4

In [40]:
data.x[1:0]

Int64[]

In [41]:
n = length(data.x)

B_jack = zeros(n)
M_jack = zeros(n)

for i in 1:n
    x_sub = append!(data.x[1:(i-1)], data.x[(i+1):n])
    y_sub = append!(data.y[1:(i-1)], data.y[(i+1):n])
    s_sub = append!(data.sigma_y[1:(i-1)], data.sigma_y[(i+1):n])
    result = optimize(p -> objective_2(x_sub, p[1], p[2], y_sub, s_sub),
                      [b,m] .+ 0.);
    @assert(Optim.converged(result))
    b,m = Optim.minimizer(result)
    B_jack[i] = b
    M_jack[i] = m
end

In [42]:
M_jack

16-element Vector{Float64}:
 2.2325412289985715
 2.3921177393926705
 2.260569143597359
 2.187504280804112
 2.2244344117718557
 2.2465137694783763
 2.2416172167077573
 2.2722680270007203
 2.238236709538266
 2.212216409243849
 2.2401279160232823
 2.238082915453938
 2.244250527384016
 2.253701257874924
 2.2047766656838403
 2.232655850273627

In [43]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
for i in 1:n
    lines!(xx, B_jack[i] .+ M_jack[i] .* xx, color=:grey)
end
lines!(xx, b_opt .+ m_opt .* xx, color=:red, linewidth=3)
f

In [44]:
f = Figure()
Axis(f[1, 1])
scatter!(B_jack, M_jack)
f

In [45]:
using Statistics

In [46]:
m_jack = mean(M_jack)
b_jack = mean(B_jack)
b_jack, m_jack

(33.029612799993025, 2.2451008793266976)

In [47]:
sigma_m_jack = (n-1)/n * std(M_jack .- m_jack)

0.04162170892310427

In [48]:
sigma_b_jack = (n-1)/n * std(B_jack .- b_jack)

7.3186580425127135

In [49]:
f = Figure(resolution=(800,500))
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)

lines!(xx, (b_jack .+ sigma_b_jack) .+ m_jack .* xx, color=:grey)
lines!(xx, (b_jack .- sigma_b_jack) .+ m_jack .* xx, color=:grey)
lines!(xx, b_jack .+ (m_jack .+ sigma_m_jack) .* xx, color=:grey)
lines!(xx, b_jack .+ (m_jack .- sigma_m_jack) .* xx, color=:grey)

lines!(xx, b_opt .+ m_opt .* xx, color=:red)#, linewidth=3)
f

In [50]:
# In the lecture

In [51]:
function objective_abs(params, x, y, sigma)
    b = params[1]
    m = params[2]
    
    y_pred = b .+ m .* data.x
    
    return -sum(-log.(sigma * sqrt(2 * π)) .-0.5 .* (y .- y_pred).^2 / sigma.^2)
    #return -gaussian_log_likelihood(data.y .+ 0., y_pred, data.sigma_y .+ 0.)
end

objective_abs (generic function with 2 methods)

In [52]:
function shim(p)
    return objective_abs(p, data.x, data.y, data.sigma_y)
end

shim (generic function with 1 method)

In [53]:
starting_params = [b , m] .+ 0.
result = optimize(p -> objective_abs(p, data.x, data.y, data.sigma_y), starting_params) # anonymous function, lambda in Python, & in Mathematica
b_gauss, m_gauss = Optim.minimizer(result)

2-element Vector{Float64}:
 28.843187456089268
  2.216658249282887

In [54]:
f = Figure()

Axis(f[1, 1])

errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)

yy_abs = m_abs.*xx .+ b_abs
lines!(xx, yy_abs, color=:green)

yy = m.*xx .+ b
lines!(xx, yy, color=:orange)

yy_gauss = m_gauss.*xx .+ b_gauss
lines!(xx, yy_gauss, color=:blue)

f

In [55]:
# Afternoon tutorial

In [56]:
ndata = size(data,1);

M_jack = zeros(ndata);
B_jack = zeros(ndata);

In [64]:
function objective_abs(params, x, y, sigma)
    b = params[1]
    m = params[2]
    
    y_pred = b .+ m .* x
    
    return -sum(-log.(sigma * sqrt(2 * π)) .-0.5 .* (y .- y_pred).^2 / sigma.^2)
    #return -gaussian_log_likelihood(data.y .+ 0., y_pred, data.sigma_y .+ 0.)
end

objective_abs (generic function with 2 methods)

In [66]:
for i in 1:ndata
    
    xcopy = copy(data.x)
    deleteat!(xcopy, i)

        
    ycopy = copy(data.y)
    deleteat!(ycopy, i)
    
    scopy = copy(data.sigma_y)
    deleteat!(scopy, i)

    starting_params = [b , m] .+ 0.
    result = optimize(p -> objective_abs(p, xcopy, ycopy, scopy), starting_params)
    
    #@assert Optim.converged(result)
    b_jack, m_jack = Optim.minimizer(result)
    
    B_jack[i] = b_jack
    M_jack[i] = m_jack
end


In [67]:
@show data.x;

data.x = [203, 58, 210, 202, 198, 158, 165, 201, 157, 131, 166, 160, 186, 125, 218, 146]


In [68]:
starting_params = [b , m] .+ 0.
result = optimize(p -> objective_abs(p, data.x, data.y, data.sigma_y), starting_params) # anonymous function, lambda in Python, & in Mathematica

 * Status: success

 * Candidate solution
    Final objective value:     1.044426e+03

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    32
    f(x) calls:    65


In [69]:
f = Figure()

Axis(f[1, 1])

errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)

for i in 1:ndata
    yy_jack = xx .* M_jack[i] .+ B_jack[i]
    lines!(xx, yy_jack, color=:green)
end

f

In [72]:
f = Figure()

Axis(f[1, 1])


scatter!(B_jack, M_jack, markersize = 10, color = :red)

plot!([b_gauss], [m_gauss], markersize = 15)

f

In [74]:
using Statistics

In [75]:
mean(B_jack), std(B_jack) .* sqrt((ndata-1) / ndata)

(27.983208497267857, 8.578829913871289)

In [76]:
mean(M_jack), std(M_jack) .* sqrt((ndata-1) / ndata)

(2.22125642553986, 0.04880410001099796)

In [77]:
f = Figure()

Axis(f[1, 1])

errorbars!(alldata.x, alldata.y, alldata.sigma_y)
scatter!(alldata.x, alldata.y, markersize = 10, color = :red)

#for i in 1:ndata
#    yy_jack = xx .* M_jack[i] .+ B_jack[i]
#    lines!(xx, yy_jack, color=:green)
#end

f

In [80]:
starting_params = [b , m] .+ 0.
result = optimize(p -> objective_abs(p, alldata.x, alldata.y, alldata.sigma_y), starting_params)
@assert Optim.converged(result)
b_bad, m_bad = Optim.minimizer(result)

2-element Vector{Float64}:
 310.0158188622038
   0.6320185504723868

In [82]:
f = Figure()

Axis(f[1, 1])

errorbars!(alldata.x, alldata.y, alldata.sigma_y)
scatter!(alldata.x, alldata.y, markersize = 10, color = :red)

yy_bad = xx .* m_bad .+ b_bad
lines!(xx, yy_bad, color=:purple, linewidth=3)

f

In [91]:
function objective_outliers(params, x, y, sigma)
    b = params[1]
    m = params[2]
    
    frac_bad = 0.01
    like_bad = frac_bad * (1. /600.)
    #loglike_bad = log(like_bad)
    
    y_pred = b .+ m .* x
    
    like_good = (1. - frac_bad)  .+ 1. ./(sigma * sqrt(2 * π)) .* exp.(-0.5 .* (y .- y_pred).^2 ./ sigma.^2)
    
    #loglike_good = log(1. - frac_bad)  + -log.(sigma * sqrt(2 * π)) .-0.5 .* (y .- y_pred).^2 / sigma.^2
    like = like_bad .+ like_good
    
    loglike = log.(like)
    
    return -sum(loglike)
    #return -gaussian_log_likelihood(data.y .+ 0., y_pred, data.sigma_y .+ 0.)
end

objective_outliers (generic function with 1 method)

In [92]:
starting_params = [b , m] .+ 0.
result = optimize(p -> objective_outliers(p, alldata.x, alldata.y, alldata.sigma_y), starting_params)
@assert Optim.converged(result)
b_out, m_out = Optim.minimizer(result)

2-element Vector{Float64}:
 35.862815414313204
  2.2648982478512636

In [95]:
starting_params = [700 , -1/2] .+ 0.
result = optimize(p -> objective_outliers(p, alldata.x, alldata.y, alldata.sigma_y), starting_params)
@assert Optim.converged(result)
b_pnk, m_pnk = Optim.minimizer(result)

2-element Vector{Float64}:
 608.6831006407621
  -0.5319722823965093

In [97]:
f = Figure()

Axis(f[1, 1])

errorbars!(alldata.x, alldata.y, alldata.sigma_y)
scatter!(alldata.x, alldata.y, markersize = 10, color = :red)

yy_bad = xx .* m_bad .+ b_bad
lines!(xx, yy_bad, color=:purple, linewidth=3)

yy_out = xx .* m_out .+ b_out
lines!(xx, yy_out, color=:green, linewidth=3)

yy_pnk = xx .* m_pnk .+ b_pnk
lines!(xx, yy_pnk, color=:red, linewidth=3)

f